# Fitting a Gaussian process kernel

In the [previous post](TODO: URL) we introduced the Gaussian process model with the exponentiated quadratic covariance function. In this post we will introduce parametrized covariance functions (kernels), fit them to real world data, and use them to make posterior predictions.

We will implement the Gaussian process model in [TensorFlow Probability](https://www.tensorflow.org/probability/) which will allow us to easily implement and tune our model without having to worry about the details.

In [1]:
# Imports
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp

import bokeh
import bokeh.io
import bokeh.plotting
import bokeh.models
from IPython.display import display, HTML
bokeh.io.output_notebook(hide_banner=True)

tfd = tfp.distributions
psd_kernels = tfp.positive_semidefinite_kernels

np.random.seed(42)
tf.set_random_seed(42)
#

In [2]:
from polynomial import Linear
from rational_quadratic import RationalQuadratic
psd_kernels.Linear = Linear
psd_kernels.RationalQuadratic = RationalQuadratic

## Mauna Loa CO2 data

The dataset used in this example is the monthly average [atmospheric CO<sub>2</sub>](https://en.wikipedia.org/wiki/Carbon_dioxide_in_Earth%27s_atmosphere) concentrations (in parts per million (ppm)) collected at the [Mauna Loa Observatory](https://en.wikipedia.org/wiki/Mauna_Loa_Observatory) in Hawaii. The observatory has been collecting these CO<sub>2</sub> concentrations since 1958 and [showed](https://en.wikipedia.org/wiki/Keeling_Curve) the first significant evidence of rapidly increasing CO<sub>2</sub> levels in the atmosphere. 

These measures of atmospheric CO<sub>2</sub> concentrations show different characteristics such as a long term rising trend, variation with the seasons, and smaller irregularities. This made it into a canonical example in Gaussian process modelling [[1](#References)].

In this post the data is downloaded as csv from the [Scripps CO<sub>2</sub> Program website](http://scrippsco2.ucsd.edu/data/atmospheric_co2/mlo). This data is loaded in a pandas dataframe and plotted below.

In [3]:
# Load the data
# Load the data from the Scripps CO2 program website. 
co2_df = pd.read_csv(
    ('http://scrippsco2.ucsd.edu/assets/data/atmospheric/stations/'
     'in_situ_co2/monthly/monthly_in_situ_co2_mlo.csv'), 
    header=54, # Data starts here
    skiprows=[55, 56], # Headers consist of multiple rows
    usecols=[3, 4], # Only keep the 'Date' and 'CO2' columns
    na_values='-99.99',  # NaNs are denoted as '-99.99'
    dtype=np.float64
)

# Drop missing values
co2_df.dropna(inplace=True)
# Remove whitespace from column names
co2_df.rename(columns=lambda x: x.strip(), inplace=True)
#

In [4]:
# Plot data
fig = bokeh.plotting.figure(
    width=600, height=300, 
    x_range=(1958, 2020), y_range=(310, 420))
fig.xaxis.axis_label = 'Date'
fig.yaxis.axis_label = 'CO₂ (ppm)'
fig.add_layout(bokeh.models.Title(
    text='In situ air measurements at Mauna Loa, Observatory, Hawaii',
    text_font_style="italic"), 'above')
fig.add_layout(bokeh.models.Title(
    text='Atmospheric CO₂ concentrations', 
    text_font_size="14pt"), 'above')
fig.line(
    co2_df.Date, co2_df.CO2, legend='All data',
    line_width=2, line_color='midnightblue')
fig.legend.location = 'top_left'
fig.toolbar.autohide = True
bokeh.plotting.show(fig)
#

In [5]:
# Split the data into observed and to predict
date_split_predict = 2008
df_observed = co2_df[co2_df.Date < date_split_predict]
print('{} measurements in the observed set'.format(len(df_observed)))
df_predict = co2_df[co2_df.Date >= date_split_predict]
print('{} measurements in the test set'.format(len(df_predict)))
#

593 measurements in the observed set
130 measurements in the test set


## Gaussian process model

We're going to use a Gaussian process model to make posterior predictions of the atmospheric CO2 concentrations for 2008 and after based on the oberserved data from before 2008.

A Gaussian process is uniquely defined by it's mean function $m(x)$ and covariance function $k(x,x')$:

$$f(x) \sim \mathcal{GP}(m(x),k(x,x'))$$

### Mean function

Since most interesting effects will be modelled by the kernel function we will keep the mean function simple. In this example the mean function is going to be modelled as a function that always returns the mean of the obeservations.

In [6]:
# Define mean function which is the means of observations
observations_mean = tf.constant(
    [np.mean(df_observed.CO2.values)], dtype=tf.float64)
mean_fn = lambda _: observations_mean

### Kernel function

To model the different characteristics of our dataset we will create a covariance (kernel) function by combining different kernels already available in [TensorFlow-Probability](https://www.tensorflow.org/probability/api_docs/python/tfp/positive_semidefinite_kernels). The different data characterics will be modelled as:

- Smooth change in CO2 levels over time modelled by a [exponentiated quadratic](https://www.tensorflow.org/probability/api_docs/python/tfp/positive_semidefinite_kernels/ExponentiatedQuadratic) kernel defined in the code below as `smooth_kernel`.
- Long term rising trend modelled a local linear kernel which consists of a linear kernel multiplied with a exponentiated quadratic to make the trend degrade as further it gets from the observations. The long term rising trend kernel is defined in the code below as `local_trend_kernel`.
- Seasonality based on a local periodic kernel, which consists of a [exponentiated sine squared kernel](https://www.tensorflow.org/probability/api_docs/python/tfp/positive_semidefinite_kernels/ExpSinSquared) multiplied with a exponentiated quadratic to make the seasonality degrade as further it gets from the observations. This seasonal periodic kernel is defined in the code below as `local_periodic_kernel`.
- Short to medium term irregularities modelled by a rational quadratic kernel, which is defined in the code below as `irregular_kernel`.
- Observational noise which will be modelled directly by the `observation_noise_variance` parameters of the [TensorFlow Gaussian process model](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/GaussianProcess).

These different kernels will be summed into one single kernel function $k_{\theta}(x_a, x_b)$ that will allow for all these effects to occur together. This kernel is defined as `kernel` in the code below. Each of the kernels has hyperparameters $\theta$ that can be tuned, they will be defined as a variable so they can be fitted on the observed data. [This post](TODO: link) provides more insight on the kernels used here and the effect of their hyperparameters.

In [7]:
# Define the kernel with trainable parameters. 
# Note we transform some of the trainable variables to ensure
#  they stay positive.

# Use float64 because this means that the kernel matrix will have 
#  less numerical issues when computing the Cholesky decomposition

# Smooth kernel hyperparameters
smooth_amplitude = tf.exp(tf.Variable(
    np.float64(3)), name='smooth_amplitude')
smooth_length_scale = tf.exp(tf.Variable(
    np.float64(3)), name='smooth_length_scale')
# Smooth kernel
smooth_kernel = psd_kernels.ExponentiatedQuadratic(
    amplitude=smooth_amplitude, 
    length_scale=smooth_length_scale)

# Local trend kernel hyperparameters
trend_slope_variance = tf.exp(tf.Variable(
    np.float64(1)), name='trend_slope_variance')
trend_bias_variance = tf.exp(tf.Variable(
    np.float64(0)), name='trend_bias_variance')
# Set initial intercept around mean of observed dates
trend_intercept = tf.Variable(
    np.float64(np.mean(df_observed.Date.values)), name='trend_intercept')
trend_local_length_scale = tf.exp(tf.Variable(
    np.float64(2)), name='trend_local_length_scale')
# Local trend kernel
local_trend_kernel = (
    psd_kernels.Linear(
        slope_variance=trend_slope_variance,
        bias_variance=trend_bias_variance,
        intercept=trend_intercept) *
    psd_kernels.ExponentiatedQuadratic( 
        length_scale=trend_local_length_scale))

# Local periodic kernel hyperparameters
periodic_amplitude = tf.exp(tf.Variable(
    np.float64(0)), name='periodic_amplitude')
periodic_length_scale = tf.exp(tf.Variable(
    np.float64(1)), name='periodic_length_scale')
periodic_period = tf.exp(tf.Variable(
    np.float64(0)), name='periodic_period')
periodic_local_length_scale = tf.exp(tf.Variable(
    np.float64(2)), name='periodic_local_length_scale')
# Local periodic kernel
local_periodic_kernel = (
    psd_kernels.ExpSinSquared(
        amplitude=periodic_amplitude, 
        length_scale=periodic_length_scale,
        period=periodic_period) * 
    psd_kernels.ExponentiatedQuadratic(
        length_scale=periodic_local_length_scale))

# Short-medium term irregularities kernel hyperparameters
irregular_amplitude = tf.exp(tf.Variable(
    np.float64(0)), name='irregular_amplitude')
irregular_length_scale = tf.exp(tf.Variable(
    np.float64(0)), name='irregular_length_scale')
irregular_scale_mixture = tf.exp(tf.Variable(
    np.float64(0)), name='irregular_scale_mixture')
# Short-medium term irregularities kernel
irregular_kernel = RationalQuadratic(
    amplitude=irregular_amplitude,
    length_scale=irregular_length_scale,
    scale_mixture_rate=irregular_scale_mixture)

# Sum all kernels to single kernel containing all characteristics
kernel = (smooth_kernel + local_periodic_kernel + irregular_kernel)

# Noise variance of observations
# Start out with a medium-to high noise
observation_noise_variance = tf.exp(tf.Variable(
    np.float64(1)), name='observation_noise_variance')
#

### Tuning the hyperparameters

We can tune the hyperparameters $\theta$ of our Gaussian process model based on the data. This post leverages TensorFlow to fit the parameters by maximizing the marginal likelihood  $p(\mathbf{y} \mid X, \theta)$ of the Gaussian process distribution based on the observed data $(X, \mathbf{y})$.

$$\hat{\theta}  = \underset{\theta}{\text{argmax}} \left( p(\mathbf{y} \mid X, \theta) \right)$$

The marginal likelihood of the Gaussian process is the likelihood of a [Gaussian distribution](TODO: link) which is defined as:

$$p(\mathbf{y} \mid \mu, \Sigma) = \frac{1}{\sqrt{(2\pi)^d \lvert\Sigma\rvert}} \exp{ \left( -\frac{1}{2}(\mathbf{y} - \mu)^{\top} \Sigma^{-1} (\mathbf{y} - \mu) \right)}$$

The mean and covariance are calulated from their parameterized functions using the observed dat $X$ as input: $\mu_{\theta} = m_{\theta}(X)$ and $\Sigma_{\theta} = k_{\theta}(X, X)$, so we can write the marginal likelihood as:

$$
p(\mathbf{y} \mid X, \theta) =
\frac{1}{\sqrt{(2\pi)^d \lvert \Sigma_{\theta} \rvert}} \exp{ \left( -\frac{1}{2}(\mathbf{y} - \mu_{\theta})^\top \Sigma_{\theta}^{-1} (\mathbf{y} - \mu_{\theta}) \right)}$$

With $d$ the dimensionality of the marginal and $\lvert \Sigma_{\theta} \rvert$ the deteriminant of the kernel matrix. We can get rid of the exponent by taking the log and maximizing the log marginal likilihood:

$$ \log{p(\mathbf{y} \mid X, \theta)} =
-\frac{1}{2}(\mathbf{y} - \mu_{\theta})^\top \Sigma_{\theta}^{-1} (\mathbf{y} - \mu_{\theta}) - \frac{1}{2} \log{\lvert \Sigma_{\theta} \rvert} - \frac{d}{2} \log{2 \pi}$$

The first term ($-0.5 (\mathbf{y} - \mu_{\theta})^\top \Sigma_{\theta}^{-1} (\mathbf{y} - \mu_{\theta})$) is the data-fit while the rest ($-0.5(\log{\lvert \Sigma_{\theta} \rvert} + d\log{2 \pi})$) is a complexity penalty, also known as [differential entropy](https://en.wikipedia.org/wiki/Differential_entropy) [[1](#Side-notes)].



The optimimal parameters $\hat{\theta}$ can then be found by minimizing the negative of the log marginal likilihood:

$$
\hat{\theta}  = \underset{\theta}{\text{argmax}} \left( p(\mathbf{y} \mid X, \theta) \right) = \underset{\theta}{\text{argmin}} { \;\log{ p(\mathbf{y} \mid X, \theta)}}$$

Since in this case the log marginal likilihood is derivable with respect to the kernel hyperparameters, we can use a [gradient based approach](TODO link) to minimize the negative log marginal likilihood (NLL). In this post we will be using a gradient descent based approach to train the hyperparameters on minibatches of the observed data.

In [8]:
# Define mini-batch data iterator
batch_size = 128

batch_date, batch_co2 = tf.data.Dataset.from_tensor_slices(
    (df_observed.Date.values.reshape(-1, 1), 
     df_observed.CO2.values - observations_mean)).shuffle(
    buffer_size=len(df_observed)).repeat(count=None).batch(
    batch_size).make_one_shot_iterator().get_next()
#

/home/peter/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


We will represent the Gaussian process marginal distribution by the [`GaussianProcess`](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/GaussianProcess) object which has a `log_prob` function to get the marginal log likelihood. The [`AdamOptimizer`](https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer) is used to minimize the negative marginal log likelihood.

In [9]:
# Gaussian process with batch data to fit the kernel parameters
gp_batched = tfd.GaussianProcess(
    mean_fn=mean_fn,
    kernel=kernel,
    index_points=batch_date,
    observation_noise_variance=observation_noise_variance)

# NLL to minimize
neg_log_likelihood_batch = -gp_batched.log_prob(batch_co2)
# Adam optimizer to minimize NLL
optimize = tf.train.AdamOptimizer(
    learning_rate=0.002).minimize(neg_log_likelihood_batch)

In [10]:
# Gaussian process on all training data to monitor NLL
#  on full observed dataset.
gp_all_observed = tfd.GaussianProcess(
    mean_fn=mean_fn,
    kernel=kernel,
    index_points=df_observed.Date.values.reshape(-1, 1),
    observation_noise_variance=observation_noise_variance)
log_likelihood_all = -gp_all_observed.log_prob(
    df_observed.CO2.values)
#

In [11]:
# Fit hyperparameters

# Start session
session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())

# Training loop
batch_nlls = []  # Batch NLL for plotting
full_ll = []  # Full data NLL for plotting
nb_iterations = 15001
for i in range(nb_iterations):
    # Run optimization for single batch
    _, nlls = session.run([optimize, neg_log_likelihood_batch])
    batch_nlls.append((i, nlls))
    # Evaluate on all observations
    if i % 100 == 0:
        # Evaluate on all observed data
        ll = session.run(log_likelihood_all)
        full_ll.append((i, ll))
#

In [12]:
# Plot NLL over iterations
fig = bokeh.plotting.figure(
    width=600, height=400, 
    x_range=(0, nb_iterations), y_range=(0, 200))
fig.add_layout(bokeh.models.Title(
    text='Negative Log-Likelihood (NLL) during training', 
    text_font_size="14pt"), 'above')
fig.xaxis.axis_label = 'iteration'
fig.yaxis.axis_label = 'NLL batch'
# First plot
fig.line(
    *zip(*batch_nlls), legend='Batch data',
    line_width=2, line_color='midnightblue')
# Seoncd plot
# Setting the second y axis range name and range
fig.extra_y_ranges = {
    'fig1ax2': bokeh.models.Range1d(start=100, end=200)}
fig.line(
    *zip(*full_ll), legend='All observed data',
    line_width=2, line_color='red', y_range_name='fig1ax2')
# Adding the second axis to the plot.  
fig.add_layout(bokeh.models.LinearAxis(
    y_range_name='fig1ax2', axis_label='NLL all'), 'right')

fig.legend.location = 'top_right'
fig.toolbar.autohide = True
bokeh.plotting.show(fig)
#

Despite the complexity term in the log marginal likilihood it is still possible to overfit to the data. While we didn't prevent further overfitting in this post, you could prevent overfitting by adding a regularization term on the hyperparameters [[2](#Side-notes)], or split the observations in training an validation sets and select the best fit on your validation data from models trained on the training data.

In [13]:
# Show values of parameters found
variables = [
    smooth_amplitude,
    smooth_length_scale,
    trend_slope_variance,
    trend_bias_variance,
    trend_intercept,
    trend_local_length_scale,
    periodic_amplitude,
    periodic_length_scale,
    periodic_period,
    periodic_local_length_scale,
    irregular_amplitude,
    irregular_length_scale,
    irregular_scale_mixture,
    observation_noise_variance
]
variables_eval = session.run(variables)

data = list([
    (var.name[:-2], var_eval) 
    for var, var_eval in zip(variables, variables_eval)])
df_variables = pd.DataFrame(
    data, columns=['Hyperparameters', 'Value'])
display(HTML(df_variables.to_html(
    index=False, float_format=lambda x: f'{x:.4f}')))
#

Hyperparameters,Value
smooth_amplitude,255.3465
smooth_length_scale,36.0583
trend_slope_variance,2.7183
trend_bias_variance,1.0000
trend_intercept,1983.2588
trend_local_length_scale,7.3891
periodic_amplitude,4.7693
periodic_length_scale,2.1148
periodic_period,0.9996
periodic_local_length_scale,206.1476


### Posterior predictions

The TensorFlow `GaussianProcess` class can only represent an unconditional Gaussian process.
To make predictions by posterior inference conditional on observed data we will need to create a [`GaussianProcessRegressionModel`](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/GaussianProcessRegressionModel) with the fitted kernel, mean function and observed data.

The posterior predictions conditioned on the observed data before 2008 are plotted in the figure below together with their 95% confidence interval. Notice that our model captures captures the different dataset characteristics such as the trend and seasonality quite well.

In [14]:
# Posterior GP using fitted kernel and observed data
gp_posterior_predict = tfd.GaussianProcessRegressionModel(
    mean_fn=mean_fn,
    kernel=kernel,
    index_points=df_predict.Date.values.reshape(-1, 1),
    observation_index_points=df_observed.Date.values.reshape(-1, 1),
    observations=df_observed.CO2.values,
    observation_noise_variance=observation_noise_variance)

# Posterior mean and standard deviation
posterior_mean_predict = gp_posterior_predict.mean() + observations_mean
posterior_std_predict = gp_posterior_predict.stddev()

In [15]:
# Plot posterior predictions

# Get posterior predictions
μ = session.run(posterior_mean_predict)
σ = session.run(posterior_std_predict)

# Plot
fig = bokeh.plotting.figure(
    width=600, height=400,
    x_range=(2008, 2019), y_range=(380, 415))
fig.xaxis.axis_label = 'Date'
fig.yaxis.axis_label = 'CO₂ (ppm)'
fig.add_layout(bokeh.models.Title(
    text='Posterior predictions conditioned on observations before 2008.',
    text_font_style="italic"), 'above')
fig.add_layout(bokeh.models.Title(
    text='Atmospheric CO₂ concentrations', 
    text_font_size="14pt"), 'above')
fig.circle(
    co2_df.Date, co2_df.CO2, legend='True data',
    size=2, line_color='midnightblue')
fig.line(
    df_predict.Date.values, μ, legend='μ (predictions)',
    line_width=2, line_color='firebrick')
# Confidence band
band_x = np.append(
    df_predict.Date.values, df_predict.Date.values[::-1])
band_y = np.append(
    (μ + 2*σ), (μ - 2*σ)[::-1])
fig.patch(
    band_x, band_y, color='firebrick', alpha=0.4, 
    line_color='firebrick', legend='2σ')

fig.legend.location = 'top_left'
fig.toolbar.autohide = True
bokeh.plotting.show(fig)
#

This post illustrated using TensorFlow probability to combine multiple out-of-the-box kernels and fit their hyperparameters on observed data. The fitted model was then used to make posterior predictions.

## Side-notes

1. Note that the [determinant](https://en.wikipedia.org/wiki/Determinant) $\lvert \Sigma \rvert$ is equal to the product of it's [eigenvalues](https://en.wikipedia.org/wiki/Eigenvalues_and_eigenvectors), and that $\lvert \Sigma \rvert$ can be interpreted as the [volume spanned by](https://en.wikipedia.org/wiki/Determinant#Volume_and_Jacobian_determinant) the covariance matrix $\Sigma$. Reducing $\lvert \Sigma \rvert$ will thus decrease the dispersion of the points coming from the distribution with covariance matrix $\Sigma$ and reduce the complexity.
2. Using TensorFlow Probability [distributions](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions) you can define a prior distribution for each hyperparameter and use the log-likelihood of this distribution as a regularization term on your parameter.

## References

1. [Gaussian Processes for Machine Learning. Chapter 5: Model Selection and Adaptation of Hyperparameters](http://www.gaussianprocess.org/gpml/chapters/RW5.pdf) by Carl Edward Rasmussen and Christopher K. I. Williams.
2. [Gaussian Processes for Regression](https://papers.nips.cc/paper/1048-gaussian-processes-for-regression.pdf) by Christopher K. I. Williams and Carl Edward Rasmussen.

In [16]:
# Versions used
print('Python: {}.{}.{}'.format(*sys.version_info[:3]))
print('Numpy: {}'.format(np.__version__))
print('Pandas: {}'.format(pd.__version__))
print('TensorFlow: {}'.format(tf.__version__))
print('TensorFlow Probability: {}'.format(tfp.__version__))
print('Bokeh: {}'.format(bokeh.__version__))
#

Python: 3.6.6
Numpy: 1.15.1
Pandas: 0.23.4
TensorFlow: 1.12.0
TensorFlow Probability: 0.5.0
Bokeh: 1.0.1


To read:
- http://130.243.105.49/Research/Learning/courses/ml/2011/lectures/ML_2011_L05.pdf 
- https://www.inf.ed.ac.uk/teaching/courses/mlpr/2016/notes/w7c_gaussian_process_kernels.pdf
- https://george.readthedocs.io/en/latest/user/kernels/
- http://ml.dcs.shef.ac.uk/gpss/gpws14/KernelDesign.pdf
